In [ ]:
#!pip install -q git+ssh://git@github.com/huggingface/transformers.git
#!pip install -U git+ssh://git@github.com/huggingface/accelerate.git
#!pip install --upgrade bitsandbytes
#!pip install trl

In [1]:
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer

In [2]:
train_dataset = load_dataset("tatsu-lab/alpaca", split="train")

Found cached dataset parquet (/media/whaow/.cache/huggingface/datasets/tatsu-lab___parquet/tatsu-lab--alpaca-2b32f0433506ef5f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [3]:
model_ckpt = "Salesforce/xgen-7b-8k-base"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


In [4]:
model = AutoModelForCausalLM.from_pretrained(
        model_ckpt, load_in_4bit=True, torch_dtype=torch.float16, device_map="auto"
    )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50313, 4096)

In [6]:
model = prepare_model_for_int8_training(model)

/home/whaow/anaconda3/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [8]:
peft_config = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")
model = get_peft_model(model, peft_config)

In [22]:
training_args = TrainingArguments(
        output_dir="xgen-7b-tuned-alpaca-l1",
        per_device_train_batch_size=4,
        optim="adamw_torch",
        logging_steps=10,
        learning_rate=2e-4,
        fp16=True,
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
        num_train_epochs=1,
        save_strategy="epoch",
        push_to_hub=False,
    )

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_args,
    packing=True,
    peft_config=peft_config,
)
trainer.train()

Step,Training Loss
10,1.530000
20,1.482700
30,1.505400
40,1.454100
50,1.424400
60,1.402700
70,1.383000
80,1.324800
90,1.301900
100,1.302000
